In [3]:
# 导入sqlite3库
import sqlite3

# 连接到测试数据库(如果不存在则创建)
conn = sqlite3.connect("test_db.db")
cursor = conn.cursor()

# 创建示例表
cursor.execute("""
    CREATE TABLE IF NOT EXISTS employees (
        id INTEGER PRIMARY KEY,
        name TEXT,
        department TEXT,
        salary INTEGER
    )
""")

# 插入示例数据
sample_data = [
    (6, "黄佳", "销售", 50000),
    (7, "宁宁", "工程", 75000),
    (8, "谦谦", "销售", 60000),
    (9, "悦悦", "工程", 80000),
    (10, "黄仁勋", "市场", 55000)
]
cursor.executemany("INSERT INTO employees VALUES (?, ?, ?, ?)", sample_data)
conn.commit()

In [4]:
from openai import OpenAI

client = OpenAI(
    api_key = 'sk-c1db2364185547f7a803a879d987d8e2',
    base_url = 'https://api.deepseek.com/v1'
)

In [5]:
# 获取数据库Schema
schema = cursor.execute("PRAGMA table_info(employees)").fetchall()
schema_str = "CREATE TABLE EMPLOYEES (\n" + "\n".join([f"{col[1]} {col[2]}" for col in schema]) + "\n)"
print("数据库Schema:")
print(schema_str)

# 定义一个函数,将查询发送给Claude并获取响应


数据库Schema:
CREATE TABLE EMPLOYEES (
id INTEGER
name TEXT
department TEXT
salary INTEGER
)


In [10]:
def ask_deepseek(query, schema):
    prompt = f"""这是一个数据库的Schema:

{schema}

根据这个Schema,你能输出一个SQL查询来回答以下问题吗?只输出SQL查询,不要输出其他任何内容。

问题:{query}
"""

    response = client.chat.completions.create(
        model='deepseek-reasoner',
        max_tokens=2048,
        messages=[{
            "role": 'user', "content":  prompt
        }]
    )
    return response.choices[0].message

In [11]:
# 示例自然语言问题
question = "工程部门员工的姓名和工资是多少?"

# 将问题发送给Claude并获取SQL查询
sql_query = ask_deepseek(question, schema_str)
print("生成的SQL查询:")
print(sql_query)

生成的SQL查询:
ChatCompletionMessage(content="SELECT name, salary FROM EMPLOYEES WHERE department = '工程部门';", refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning_content="首先，用户提供了一个数据库Schema：\n\nCREATE TABLE EMPLOYEES (\nid INTEGER\nname TEXT\ndepartment TEXT\nsalary INTEGER\n)\n\n问题：工程部门员工的姓名和工资是多少？\n\n我需要输出一个SQL查询来回答这个问题。只输出SQL查询，不要输出其他任何内容。\n\n查询应该从EMPLOYEES表中选择姓名和工资，其中部门是“工程部门”。在SQL中，我们使用SELECT语句来指定要检索的列，使用WHERE子句来过滤记录。\n\n列名是：id, name, department, salary。所以，我需要选择name和salary。\n\n部门是“工程部门”，所以在WHERE子句中，我应该使用department = '工程部门'。\n\n注意：部门值在Schema中是TEXT类型，所以应该用单引号括起来。\n\nSQL查询应该是：\n\nSELECT name, salary FROM EMPLOYEES WHERE department = '工程部门';\n\n确保语法正确。在SQL中，字符串值通常用单引号表示。\n\n另外，表名是EMPLOYEES，全部大写，但在SQL中，表名和列名通常不区分大小写，但为了保持一致，我使用大写。\n\n用户说“工程部门”，在中文中，所以部门值应该是“工程部门”。在查询中，我直接使用这个字符串。\n\n最终查询：SELECT name, salary FROM EMPLOYEES WHERE department = '工程部门';\n\n现在，只输出这个查询，不要其他内容。")
